# Assignment
In this assignment, you'll be working with the U.S. Education Dataset from Kaggle. The data gives detailed state level information on several facets of the state of education on an annual basis. To learn more about the data and the column descriptions, click the Kaggle link above.

Access this data from the Thinkful database using the following credentials:

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

In [34]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [40]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df0 = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [41]:
# Import the data into two different pandas for later parallel treatments
df = df0.copy()
df1 = df0.copy()

# 1. Determine all the variable types and find the fraction of the missing values for each variable.

In [42]:
# Help functions to gather basic descriptions
def describe(df):
    return pd.concat([df.describe().T,
                      df.mad().rename('mean abs dev'),
                      df.skew().rename('skew'),
                      df.kurt().rename('kurt'),
                      df.nunique().rename('unique')
                     ], axis=1).T

In [43]:
describe(df)

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,PRIMARY_KEY,STATE
count,1492.000000,1.229000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.280000e+03,1.229000e+03,...,1361.000000,1.361000e+03,1.361000e+03,1.319000e+03,536.000000,532.000000,533.000000,498.000000,NaN,NaN
mean,2004.433646,9.159308e+05,9.092082e+06,7.663723e+05,4.216553e+06,4.109157e+06,9.196681e+06,4.762966e+06,2.680331e+06,4.292046e+05,...,54268.924320,5.192140e+05,2.470714e+05,8.024415e+05,234.768293,278.414711,218.866154,263.661132,NaN,NaN
std,7.393983,1.065280e+06,1.174519e+07,1.145242e+06,5.543072e+06,5.482971e+06,1.197813e+07,6.293004e+06,3.353349e+06,5.340693e+05,...,67889.276230,6.436384e+05,3.077056e+05,9.707031e+05,10.221511,10.159722,7.769616,6.803725,NaN,NaN
min,1992.000000,4.386600e+04,4.656500e+05,3.102000e+04,0.000000e+00,2.209300e+04,4.816650e+05,2.655490e+05,1.399630e+05,1.154100e+04,...,311.000000,4.878000e+03,1.808000e+03,7.254000e+03,187.134670,232.831510,178.557612,236.379102,NaN,NaN
25%,1998.000000,2.583140e+05,2.186305e+06,1.893540e+05,1.153097e+06,7.158345e+05,2.165404e+06,1.168032e+06,6.357900e+05,1.028310e+05,...,10919.000000,1.106260e+05,5.147100e+04,1.810650e+05,229.694352,272.761598,214.663401,259.533171,NaN,NaN
50%,2004.000000,6.483130e+05,5.079546e+06,4.033765e+05,2.537074e+06,2.055780e+06,5.234506e+06,2.657452e+06,1.525406e+06,2.715960e+05,...,36735.000000,3.457750e+05,1.642600e+05,5.503420e+05,237.238552,280.618803,220.416034,265.010912,NaN,NaN
75%,2011.000000,1.014528e+06,1.085985e+07,8.289660e+05,5.080939e+06,4.768680e+06,1.074519e+07,5.568028e+06,3.229651e+06,5.186000e+05,...,67460.000000,6.119050e+05,2.905020e+05,9.282755e+05,241.995486,285.347428,223.999337,268.190121,NaN,NaN
max,2017.000000,6.307022e+06,8.921726e+07,9.990221e+06,5.090457e+07,3.610526e+07,8.532013e+07,4.396452e+07,2.605802e+07,3.995951e+06,...,498403.000000,3.929869e+06,2.013687e+06,5.944746e+06,253.420961,300.568235,236.773867,280.499130,NaN,NaN
mean abs dev,6.387714,6.705358e+05,7.546454e+06,6.488296e+05,3.343492e+06,3.706985e+06,7.638374e+06,3.976329e+06,2.207613e+06,3.363081e+05,...,42820.966192,4.040466e+05,1.946200e+05,6.092893e+05,7.983735,7.862890,5.984936,5.392384,NaN,NaN
skew,0.001469,2.887208e+00,3.017192e+00,4.157090e+00,3.773425e+00,2.551894e+00,3.016692e+00,3.123370e+00,2.887347e+00,3.229164e+00,...,3.307652,2.971172e+00,3.046377e+00,2.956857e+00,-1.097832,-1.030171,-1.092180,-0.816788,NaN,NaN


## Numerical data types

In [44]:
# Numerical describe
df_stat = df.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492,1229,1280,1280,1280,1280,1280,1280,1280,1229,...,1361,1361,1361,1361,1361,1319,536,532,533,498
mean,2004.43,915931,9.09208e+06,766372,4.21655e+06,4.10916e+06,9.19668e+06,4.76297e+06,2.68033e+06,429205,...,64538.9,64271.1,54268.9,519214,247071,802442,234.768,278.415,218.866,263.661
std,7.39398,1.06528e+06,1.17452e+07,1.14524e+06,5.54307e+06,5.48297e+06,1.19781e+07,6.293e+06,3.35335e+06,534069,...,80293.4,78910.5,67889.3,643638,307706,970703,10.2215,10.1597,7.76962,6.80373
min,1992,43866,465650,31020,0,22093,481665,265549,139963,11541,...,633,437,311,4878,1808,7254,187.135,232.832,178.558,236.379
25%,1998,258314,2.18631e+06,189354,1.1531e+06,715834,2.1654e+06,1.16803e+06,635790,102831,...,13739,13342,10919,110626,51471,181065,229.694,272.762,214.663,259.533
50%,2004,648313,5.07955e+06,403376,2.53707e+06,2.05578e+06,5.23451e+06,2.65745e+06,1.52541e+06,271596,...,43272,43339,36735,345775,164260,550342,237.239,280.619,220.416,265.011
75%,2011,1.01453e+06,1.08598e+07,828966,5.08094e+06,4.76868e+06,1.07452e+07,5.56803e+06,3.22965e+06,518600,...,75481,76566,67460,611905,290502,928276,241.995,285.347,223.999,268.19
max,2017,6.30702e+06,8.92173e+07,9.99022e+06,5.09046e+07,3.61053e+07,8.53201e+07,4.39645e+07,2.6058e+07,3.99595e+06,...,493415,500143,498403,3.92987e+06,2.01369e+06,5.94475e+06,253.421,300.568,236.774,280.499
dtype,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
size,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492,...,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492


### Many of columns have missing value that we will address later

## Object data type

In [45]:
# Non-numerical describe
df_stat = df.describe(include = ['O']) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

,PRIMARY_KEY,STATE
count,1492,1492
unique,1487,80
top,2008_VIRGINIA,DISTRICT_OF_COLUMBIA
freq,3,29
dtype,object,object
size,1492,1492
% Null,0,0


### Two of the columns have string data types

# 2. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

#### Assuming no timeseries effect here, then following strategies are selected:
#### Interpolate: 'STATE',
#### Select mean: 'ENROLL', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G','GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G','GRADES_9_12_G', 'GRADES_ALL_G', 
#### Interpolate: 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE','AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'],
#### Drop: 'TOTAL_REVENUE', 'TOTAL_EXPENDITURE'
#### Custom: After having above created then we can create new Primary Key

# 3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

## Fill NAs with mean
### First we use mean within each year. And if there is no values within a year we use State as a grouping criteria


In [46]:
df.head(2)

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712


### We use groupy and transform functions to apply mean value for all missing values

In [47]:
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'float64' else x.fillna('NaN'))

In [48]:
# Numerical describe
df_stat = df.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492,...,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492
mean,2004.43,915931,9.09208e+06,766372,4.21655e+06,4.10916e+06,9.19668e+06,4.76297e+06,2.68033e+06,429205,...,64538.9,64271.1,54268.9,519214,247071,802442,234.768,278.415,218.866,263.661
std,7.39398,966773,1.08782e+07,1.0607e+06,5.13389e+06,5.07823e+06,1.10939e+07,5.82847e+06,3.10581e+06,484683,...,76685,75364.3,64838.3,614713,293877,912652,6.12284,6.06304,4.64105,3.92813
min,1992,43866,465650,31020,0,22093,481665,265549,139963,11541,...,633,437,311,4878,1808,7254,187.135,232.832,178.558,236.379
25%,1998,315094,2.54626e+06,217020,1.35643e+06,944796,2.52397e+06,1.34361e+06,767369,135428,...,15195.5,15682.8,13437,121152,59197.5,229358,234.768,278.415,218.866,263.661
50%,2004,820414,6.35931e+06,516741,3.12764e+06,2.69726e+06,6.52022e+06,3.35814e+06,1.91384e+06,367234,...,49884.5,49571,39566,401586,181701,647326,234.768,278.415,218.866,263.661
75%,2011,921178,9.27696e+06,766372,4.26981e+06,4.10916e+06,9.43492e+06,4.93882e+06,2.79589e+06,456892,...,71814.2,72131,64616.2,574912,282539,872422,234.768,278.415,218.866,263.661
max,2017,6.30702e+06,8.92173e+07,9.99022e+06,5.09046e+07,3.61053e+07,8.53201e+07,4.39645e+07,2.6058e+07,3.99595e+06,...,493415,500143,498403,3.92987e+06,2.01369e+06,5.94475e+06,253.421,300.568,236.774,280.499
dtype,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
size,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492,...,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492


In [49]:
# # fill_by_mean = ['ENROLL', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE',\
# #                 'OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G','GRADES_4_G', 'GRADES_8_G', \
# #                 'GRADES_12_G', 'GRADES_1_8_G','GRADES_9_12_G', 'GRADES_ALL_G', 
# # ]

# fill_by_mean = df.columns
# fill_by_mean = [e for e in fill_by_mean if e not in ('YEAR')]#,'PRIMARY_KEY', 'STATE')]

# # Fill NAs with mean within each year
# for column_name in fill_by_mean:
#     df[column_name] = df.groupby("YEAR").transform(lambda x: x.fillna(x.mean()))


# # # Fill NAs with mean within each STATE after within year is performed
# # for column_name in fill_by_mean:
# #     df[column_name] = df.groupby("STATE").transform(lambda x: x.fillna(x.mean()))

# 4. This time, fill in the missing values using interpolation (extrapolation).

In [50]:
# Numerical describe
df_stat = df1.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492,1229,1280,1280,1280,1280,1280,1280,1280,1229,...,1361,1361,1361,1361,1361,1319,536,532,533,498
mean,2004.43,915931,9.09208e+06,766372,4.21655e+06,4.10916e+06,9.19668e+06,4.76297e+06,2.68033e+06,429205,...,64538.9,64271.1,54268.9,519214,247071,802442,234.768,278.415,218.866,263.661
std,7.39398,1.06528e+06,1.17452e+07,1.14524e+06,5.54307e+06,5.48297e+06,1.19781e+07,6.293e+06,3.35335e+06,534069,...,80293.4,78910.5,67889.3,643638,307706,970703,10.2215,10.1597,7.76962,6.80373
min,1992,43866,465650,31020,0,22093,481665,265549,139963,11541,...,633,437,311,4878,1808,7254,187.135,232.832,178.558,236.379
25%,1998,258314,2.18631e+06,189354,1.1531e+06,715834,2.1654e+06,1.16803e+06,635790,102831,...,13739,13342,10919,110626,51471,181065,229.694,272.762,214.663,259.533
50%,2004,648313,5.07955e+06,403376,2.53707e+06,2.05578e+06,5.23451e+06,2.65745e+06,1.52541e+06,271596,...,43272,43339,36735,345775,164260,550342,237.239,280.619,220.416,265.011
75%,2011,1.01453e+06,1.08598e+07,828966,5.08094e+06,4.76868e+06,1.07452e+07,5.56803e+06,3.22965e+06,518600,...,75481,76566,67460,611905,290502,928276,241.995,285.347,223.999,268.19
max,2017,6.30702e+06,8.92173e+07,9.99022e+06,5.09046e+07,3.61053e+07,8.53201e+07,4.39645e+07,2.6058e+07,3.99595e+06,...,493415,500143,498403,3.92987e+06,2.01369e+06,5.94475e+06,253.421,300.568,236.774,280.499
dtype,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
size,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492,...,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492


In [51]:
# Interpolate the missing values using Linear method. Mind this method ignores the index and treat the values as equally spaced.
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'float64' else x.interpolate(method ='linear', limit_direction ='forward') )

# 5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

## Some of the timeseries data have cyclical characteristics, making them more suitable for mean method to fill the missing values. Linear interpolations of the missing values for these data ignore such cyclical trend. 
## Other set of data such as grades are not cyclical and using mean method for missing values are more appropriate than interpolation. Interpolation for grades may risk of biasing becasue of proximity to nearby grades.